In [1]:
import io
import pandas as pd
import requests
from datetime import datetime as dt
import matplotlib.pyplot as plt

In [2]:
file_url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_180623.txt"
s = requests.get(file_url).content
turns_df = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
len(turns_df)

196686

In [4]:
turns_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [5]:
turns_df.rename(columns={column:column.strip() for column in turns_df.columns}, inplace=True)
turns_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [6]:
turns_df["DATE_TIME"] = pd.to_datetime(turns_df.DATE + " " + turns_df.TIME, format="%m/%d/%Y %H:%M:%S")
turns_stations = turns_df 
turns_stations[["REAL_ENTRIES", "REAL_EXITS"]] = (turns_stations.groupby(["C/A", "UNIT", "SCP", "STATION"])
                                                  ["ENTRIES", "EXITS"]
                                                  .transform(lambda x: x - x.shift(1)))
turns_stations.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,REAL_ENTRIES,REAL_EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,00:00:00,REGULAR,6658725,2257032,2018-06-16 00:00:00,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,04:00:00,REGULAR,6658747,2257036,2018-06-16 04:00:00,22.0,4.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,08:00:00,REGULAR,6658766,2257068,2018-06-16 08:00:00,19.0,32.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,12:00:00,REGULAR,6658866,2257139,2018-06-16 12:00:00,100.0,71.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,16:00:00,REGULAR,6659043,2257188,2018-06-16 16:00:00,177.0,49.0


In [7]:
# Percentage of NaN in real entries or real exits overall
100 * len(turns_stations[(turns_stations["REAL_ENTRIES"].isnull()) | (turns_stations["REAL_EXITS"].isnull())])/len(turns_stations)

2.3946798450321833

In [8]:
turns_stations.dropna(subset=["REAL_ENTRIES", "REAL_EXITS"], axis=0, inplace=True)
turns_stations["TOTAL_REAL"] = turns_stations["REAL_ENTRIES"] + turns_stations["REAL_EXITS"]
turns_stations.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME,REAL_ENTRIES,REAL_EXITS,TOTAL_REAL
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,04:00:00,REGULAR,6658747,2257036,2018-06-16 04:00:00,22.0,4.0,26.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,08:00:00,REGULAR,6658766,2257068,2018-06-16 08:00:00,19.0,32.0,51.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,12:00:00,REGULAR,6658866,2257139,2018-06-16 12:00:00,100.0,71.0,171.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,16:00:00,REGULAR,6659043,2257188,2018-06-16 16:00:00,177.0,49.0,226.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/16/2018,20:00:00,REGULAR,6659281,2257223,2018-06-16 20:00:00,238.0,35.0,273.0


In [9]:
# Percentage of negative entries in real entries overall
100 * len(turns_stations[turns_stations["REAL_ENTRIES"] < 0])/len(turns_stations)

0.8855273575863649

In [10]:
# Percentage of negative entries in real exits overall
100 * len(turns_stations[turns_stations["REAL_EXITS"] < 0])/len(turns_stations)

0.7381130974705171

In [11]:
# Percentage of negative entries in real entries per station
100 * (turns_stations[turns_stations["REAL_ENTRIES"] < 0].groupby(["STATION"]).size()/turns_stations.groupby(["STATION"])
 .size()).dropna().sort_values(ascending = False).head(10)

STATION
190 ST             29.166667
104 ST             22.222222
6 AV               20.000000
174 ST             16.666667
EASTCHSTER/DYRE    14.146341
BAY PKWY           13.750000
3 AV-149 ST        13.333333
57 ST               8.565310
GRAND-NEWTOWN       8.333333
METROPOLITAN AV     7.142857
dtype: float64

In [12]:
# Percentage of negative entries in real exits per station
100 * (turns_stations[turns_stations["REAL_EXITS"] < 0].groupby(["STATION"]).size()/turns_stations.groupby(["STATION"])
 .size()).dropna().sort_values(ascending = False).head(10)

STATION
225 ST           33.333333
190 ST           30.833333
AVENUE I         23.170732
104 ST           22.222222
6 AV             20.000000
174 ST           16.666667
HIGH ST          16.666667
57 ST-7 AV       14.285714
PROSPECT PARK    12.500000
75 AV            10.000000
dtype: float64

In [13]:
#Droping all negative entries  
turns_stations = (turns_stations.drop(turns_stations
                    [(turns_stations["REAL_EXITS"] < 0) | (turns_stations["REAL_ENTRIES"] < 0)].index))

In [14]:
# Where and when we have more people
(turns_stations.groupby(["STATION","DATE_TIME"])).sum().sort_values("TOTAL_REAL", ascending = False).head(20)

ENTRIES       EXITS  REAL_ENTRIES  \
STATION         DATE_TIME                                                   
CANAL ST        2018-06-18 09:00:00  5429112866  4273863454  2.105703e+09   
LEXINGTON AV/53 2018-06-19 12:00:00  2355056998  1808673956  6.319330e+05   
THIRTY THIRD ST 2018-06-18 06:25:59      337145      239224  3.371450e+05   
34 ST-HERALD SQ 2018-06-21 20:00:00  1552119904  1353681276  4.978900e+04   
                2018-06-20 20:00:00  1551998407  1353569505  5.140600e+04   
                2018-06-19 20:00:00  1551878180  1353459421  5.068300e+04   
34 ST-PENN STA  2018-06-21 20:00:00  1909588649  1709954720  3.485600e+04   
                2018-06-19 20:00:00  1909343179  1709729668  3.470800e+04   
                2018-06-20 20:00:00  1909464686  1709840091  3.501200e+04   
34 ST-HERALD SQ 2018-06-18 20:00:00  1551761983  1353352917  4.804400e+04   
14 ST-UNION SQ  2018-06-19 21:00:00   886794034   296233249  4.787700e+04   
34 ST-PENN STA  2018-06-18 20:00:00  1909223375  1709619479  3.288800e+04   
34 ST-HERALD SQ 2018-06-22 20:00:00  1552235646  1353789391  4.310400e+04   
34 ST-PENN STA  2018-06-22 20:00:00  1909702920  1710067310  3.110200e+04   
TIMES SQ-42 ST  2018-06-21 20:00:00  3946221675  3126631752  4.492900e+04   
                2018-06-20 20:00:00  3946112766  3126528531  4.618900e+04   
                2018-06-19 20:00:00  3946006073  3126426731  4.517500e+04   
                2018-06-18 20:00:00  3945903322  3126328472  4.349000e+04   
                2018-06-22 20:00:00  3946328667  3126729942  3.836900e+04   
14 ST-UNION SQ  2018-06-21 21:00:00   887011697   296418553  4.018500e+04   

                                      REAL_EXITS    TOTAL_REAL  
STATION         DATE_TIME                                       
CANAL ST        2018-06-18 09:00:00  618378596.0  2.724082e+09  
LEXINGTON AV/53 2018-06-19 12:00:00     404188.0  1.036121e+06  
THIRTY THIRD ST 2018-06-18 06:25:59     238191.0  5.753360e+05  
34 ST-HERALD SQ 2018-06-21 20:00:00      34187.0  8.397600e+04  
                2018-06-20 20:00:00      32124.0  8.353000e+04  
                2018-06-19 20:00:00      32174.0  8.285700e+04  
34 ST-PENN STA  2018-06-21 20:00:00      46701.0  8.155700e+04  
                2018-06-19 20:00:00      46464.0  8.117200e+04  
                2018-06-20 20:00:00      46007.0  8.101900e+04  
34 ST-HERALD SQ 2018-06-18 20:00:00      29812.0  7.785600e+04  
14 ST-UNION SQ  2018-06-19 21:00:00      29359.0  7.723600e+04  
34 ST-PENN STA  2018-06-18 20:00:00      43584.0  7.647200e+04  
34 ST-HERALD SQ 2018-06-22 20:00:00      31545.0  7.464900e+04  
34 ST-PENN STA  2018-06-22 20:00:00      41899.0  7.300100e+04  
TIMES SQ-42 ST  2018-06-21 20:00:00      26469.0  7.139800e+04  
                2018-06-20 20:00:00      24896.0  7.108500e+04  
                2018-06-19 20:00:00      24762.0  6.993700e+04  
                2018-06-18 20:00:00      22517.0  6.600700e+04  
                2018-06-22 20:00:00      26187.0  6.455600e+04  
14 ST-UNION SQ  2018-06-21 21:00:00      22989.0  6.317400e+04